## Environment Setting

In [2]:
from flask import Flask, request, jsonify
import cv2
import numpy as np
import tensorflow as tf
import pickle
import joblib
import sys
import os
import pandas as pd 
from keras_vggface.utils import preprocess_input
from keras_vggface import VGGFace
from keras.models import Model
from tensorflow.keras.preprocessing.image import load_img
import warnings
warnings.filterwarnings("ignore")


## Load Image

In [16]:
def preprocessing_v1(img):
    
    sample = img.copy()
    sample = cv2.resize(sample, (224, 224))
    sample = np.array(sample).astype(np.float64)
    sample = np.expand_dims(sample, axis = 0)
    sample = preprocess_input(sample, version = 1)
    
    return sample

In [17]:
def preprocessing_v2(img):
    
    sample = img.copy()
    sample = cv2.resize(sample, (224, 224))
    sample = np.array(sample).astype(np.float64)
    sample = np.expand_dims(sample, axis = 0)
    sample = preprocess_input(sample, version = 2)
    
    return sample

## Feature Extraction

In [11]:
"""
# Load the VGGFace model
vggface = VGGFace(model = 'vgg16', include_top = True, input_shape = (224, 224, 3), pooling = 'avg')

# retrieve fc6 layer
fc6 = vggface.get_layer('fc6')
new_vgg16 = Model(inputs = vggface.input, outputs = fc6.output)
"""

"\n# Load the VGGFace model\nvggface = VGGFace(model = 'vgg16', include_top = True, input_shape = (224, 224, 3), pooling = 'avg')\n\n# retrieve fc6 layer\nfc6 = vggface.get_layer('fc6')\nnew_vgg16 = Model(inputs = vggface.input, outputs = fc6.output)\n"

In [8]:
vgg16 = VGGFace(model='vgg16')
vgg16_fc6 = Model(inputs=vgg16.input, outputs=vgg16.get_layer('fc6').output)

In [9]:
vgg_resnet = VGGFace(model='resnet50')
vgg_resnet_avg_pool = Model(inputs=vgg_resnet.input, outputs=vgg_resnet.get_layer('avg_pool').output)

In [10]:
vgg_senet = VGGFace(model='senet50')
vgg_senet_avg_pool = Model(inputs=vgg_senet.input, outputs=vgg_senet.get_layer('avg_pool').output)

## Model Loading

In [7]:
vgg16_rbf_model = joblib.load('vgg16_SVR_rbf_model.pkl')

In [5]:
# Load the Regressor
senet_rbf_model = joblib.load('senet_SVR_rbf_model.pkl')

In [4]:
resnet_rbf_model = joblib.load('resnet_SVR_rbf_model.pkl')

## Prediction

In [11]:
def predict_bmi():

    picKey = ' ' # Currently set to space

    # Set cascade file path and create the face cascade
    cascPath = "haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(cascPath)

    # Set video source to default webcam
    video_capture = cv2.VideoCapture(0)

    # Initialize the count variable, used in naming frame images
    count = 0

    # Specify the font
    font = cv2.FONT_HERSHEY_SIMPLEX

    # Create the directory to store captured images
    save_dir = "D:/Uchicago/2023_Spring/MSCA-31009/Project/web_cam_capture"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    while True:
        # Capture frame-by-frame
        ret, frame = video_capture.read()

        # Detect faces
        faces = faceCascade.detectMultiScale(
            frame,
            scaleFactor=1.15,
            minNeighbors=5,
            minSize=(30, 30),
        )

        for (x, y, w, h) in faces:
            # box bounding the face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            processed_img = preprocessing_v2(frame[y:y+h, x:x+w])
            features = vgg_senet_avg_pool.predict(processed_img)
            # Flatten the features for SVR
            features_flatten = features.reshape(features.shape[0], -1)
            preds = senet_rbf_model.predict(features_flatten)
            cv2.putText(frame, f'BMI: {preds}', (x+5, y-5), font, 1, (255, 255, 255), 2)

        cv2.imshow('Predicting BMI', frame)

        # Wait for keypress and save value
        capKey = cv2.waitKey(1)

        # Take a picture if picKey is pressed
        if capKey & 0xFF == ord(picKey):
            save_path = os.path.join(save_dir, "frame{}.jpg".format(count))
            cv2.imwrite(save_path, frame)
            count += 1

        # Exit script if quitKey is pressed
        elif capKey & 0xFF == ord('q'):
            break

    # Release the video capture
    video_capture.release()
    cv2.destroyAllWindows()
